# TensorFlow实现多层感知机

In [1]:
# 加载库文件
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# 加载数据
mnist = input_data.read_data_sets("../datasets/MNIST_data", one_hot=True)

Extracting ../datasets/MNIST_data\train-images-idx3-ubyte.gz
Extracting ../datasets/MNIST_data\train-labels-idx1-ubyte.gz
Extracting ../datasets/MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ../datasets/MNIST_data\t10k-labels-idx1-ubyte.gz


In [3]:
# 定义默认的会话框
sess = tf.InteractiveSession()

In [4]:
# 构建计算图
# 定义参数
# W1、b1是隐含层的权重和偏置，我们将偏置全部置于0，并将权重初始化为截断的正太分布，其标准差为0.1
# 这一步可以通过tf.truncated_normal方便的实现。因为模型使用的激活函数是ReLU,所以需要
# 使用正太分布给参数加一点噪声，来打破完全对称并且避免0梯度。在其他一些模型中，有时还需要
# 给偏置项加上一些小的非零来避免dead neuron(死亡神经元)，不过这里作用不太明显。
# 而对最后输出层的Softmax，直接将权重W2和偏置b2全部初始化为0即可
in_units = 784
h1_units = 300
W1 = tf.Variable(tf.truncated_normal([in_units, h1_units], stddev=0.1))
b1 = tf.Variable(tf.zeros([h1_units]))
W2 = tf.Variable(tf.zeros([h1_units, 10]))
b2 = tf.Variable(tf.zeros([10]))

# Drop_out的比率keep_prob(即保留节点的概率)是不一样的，通常训练时小于1，
# 而预测时则等于1，所有把dropout也加入计算图，并定义为一个placeholder
keep_prob = tf.placeholder(tf.float32)
x = tf.placeholder(tf.float32, [None, in_units]) # 构建输入数据
y_ = tf.placeholder(tf.float32, [None, 10]) # 构建输出数据
global_step = tf.Variable(0, trainable=False, dtype=tf.int64)

# 构建前馈计算图
hidden1 = tf.nn.relu(tf.matmul(x, W1) + b1)
hidden1_drop = tf.nn.dropout(hidden1, keep_prob)
y = tf.nn.softmax(tf.matmul(hidden1_drop, W2) + b2)

# 构建算损失函数
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y),
                              reduction_indices = [1]))
train_step = tf.train.AdagradOptimizer(0.3).minimize(cross_entropy)


# 进行迭代
tf.global_variables_initializer().run()
for i in range(300):
    batch_xs, batch_ys = mnist.train.next_batch(500)
    train_step.run({x: batch_xs, y_:batch_ys, keep_prob:0.75})
    
# 进行预测
correct_prediction =tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval({x: mnist.test.images, y_:mnist.test.labels, keep_prob: 1.0}))

0.9628
